In [1]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter
s3 = var("s3", latex_name="s_3")        ## Third relaxation parameter

assume(C,'real')
assume(C>-1/2)
assume(C< 1/2)

s2 = 2
s3 = 2

#### Pieces involved in the scheme
q = 3 ## Number of discrete velocities
c1 = 0  ## First discrete velocity
c2 = 1  ## Second discrete velocity
c3 = -1 ## Third discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1,  1], \
            [0,  1, -1], \
            [0,  1,  1]]) ## Moment matrix
eps1 = 1              ## First equilibrium coefficient (useless)
eps2 = C              ## Second equilibrium coefficient
eps3 = 1/3*(1+2*C**2) ## Third equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa") 

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2, s3])\
        *(matrix([[eps1], [eps2], [eps3]])*matrix([[1, 0, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2), k**(-c3)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z") 

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = (dm1/dp1).full_simplify() ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [2]:
##### Boundary conditions that we analyze for any value of C and s_2 and s_3
bdMat_BB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 1], [0, 0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, -1], [0, 0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [-1, 0, -k], [0, 0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 1, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 2-k, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 3-3*k+k**2, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 4-6*k+4*k**2-k**3, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 0], [0, 0, k]])*Minv*K ## Kinetic dirichlet

In [3]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k))) 
print("Two steps anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [4]:
##### Sanity check : we set some values for C and see if we find more
##### Sometimes sagemath does not give all the roots

C_val = -3/13

print("Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_BB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_ABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots")
print("Two steps anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_TwoABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma1.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma2.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma3.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma4.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_kinDir.subs(C=C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1], [z == -16/507*I*sqrt(935) + 133/507, kappa == -1872/73529*I*sqrt(935) - 46151/73529], [z == 16/507*I*sqrt(935) + 133/507, kappa == 1872/73529*I*sqrt(935) - 46151/73529]]

!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots
Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [6]:
print("Anti Bounce back (now correct)")
kABB = solve(bdMat_ABB.determinant().full_simplify(), k)[0].rhs().full_simplify()
zABB = solve(charEq.full_simplify().subs(k==kABB), z)
pretty_print(zABB)

zABBMinus = zABB[0].rhs()
zABBPlus  = zABB[1].rhs()

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBMinus).full_simplify(), k))

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBPlus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBPlus).full_simplify(), k))

#print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))

R = -(4*C^4 - 2*C^2 - 2).factor()
RealPoly = 32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - 6*C + 10
ImagPoly = 16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2
ER = (1/2*(RealPoly+RealPoly.subs(C==-C))).factor()
OR = (1/2*(RealPoly-RealPoly.subs(C==-C))).factor()
EI = (1/2*(ImagPoly+ImagPoly.subs(C==-C))).factor()
OI = (1/2*(ImagPoly-ImagPoly.subs(C==-C))).factor()

print('values of E_R, O_R, E_I and O_I')
pretty_print(ER)
pretty_print(OR)
pretty_print(EI)
pretty_print(OI)

er = var("er", latex_name="\\mathcal{E}_R") 
ei = var("ei", latex_name="\\mathcal{E}_I") 
odr = var("odr", latex_name="\\mathcal{O}_R") 
odi = var("odi", latex_name="\\mathcal{O}_I") 
r = var("r", latex_name="\\mathcal{R}") 
assume(er, 'real')
assume(ei, 'real')
assume(odr, 'real')
assume(odi, 'real')
assume(r>0)


num = (er+odr+I*(ei+odi)*sqrt(r))
den = (er-odr+I*(ei-odi)*sqrt(r))

numMod = ((num*conjugate(den)).full_simplify())
denMod = ((den*conjugate(den)).full_simplify())
print('Numerator in abs(k)^2')
pretty_print((numMod.real()**2+numMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())
print('Denominator in abs(k)^2')
pretty_print((denMod.real()**2+denMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())


dz = var("dz", latex_name="\\delta z") 
dk = var("dk", latex_name="\\delta \\kappa") 

print('Perturbation test on the other root (k=-1) on the critical z-s')
pretty_print(solve(taylor(charEq.subs(z==zABBMinus*(1+dz), k==-1+dk).full_simplify(), (dz, 0), (dk, 0), 1), dk))
pretty_print(solve(taylor(charEq.subs(z==zABBPlus*(1+dz), k==-1+dk).full_simplify(), (dz, 0), (dk, 0), 1), dk))

#assume(C<0)

print('To see whether the parametrization here is kminus or kplus')
for Cval in [-1/10, -1/4, -11/23]:
    pretty_print(kminus.subs(z==zABBMinus).full_simplify().subs(C==Cval).n())
    pretty_print(kplus.subs(z==zABBMinus).full_simplify().subs(C==Cval).n())
    pretty_print(kminus.subs(z==zABBPlus).full_simplify().subs(C==Cval).n())
    pretty_print(kplus.subs(z==zABBPlus).full_simplify().subs(C==Cval).n())
    print('---')
print('Conclusion: the right parametrization is kplus')    

ks = kplus.full_simplify()

Cval = -1/4
Vgroup = -(ks/z/derivative(ks, z)).subs(z==zABBMinus).full_simplify().subs(C==Cval) ### Group velocity without \lambda
print('group velocity (without lambda)')
pretty_print(Vgroup)
print('group velocity (without lambda), totally simplified')
pretty_print(abs(Vgroup.maxima_methods().polarform()).canonicalize_radical()) ### https://ask.sagemath.org/question/43617/is-there-a-sage-command-to-compute-complex-numbers/
#### This maxima_methods().polarform()).canonicalize_radical() can be in principle be used to study the modulus of k as done before

var('n')
print('To understand what is the resonant source term for this instability')
pretty_print((zABBMinus.canonicalize_radical().maxima_methods().polarform()**n).expand().full_simplify())
pretty_print((zABBPlus.canonicalize_radical().maxima_methods().polarform()**n).expand().full_simplify())


print('Computation of the stable eigenvector')
LzOnStable = Lz.subs(k==ks)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
LzOnStableKer = LzOnStable.transpose().kernel()
phiS = matrix([[LzOnStableKer.basis()[0][0].full_simplify()], \
               [LzOnStableKer.basis()[0][1].full_simplify()], \
               [LzOnStableKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates

print('We observe that the eigenvector does not blow up')
for Cval in [-1/10, -1/4, -11/23]:
    pretty_print(phiS[1][0].subs(z==zABBMinus).subs(C==Cval).canonicalize_radical().full_simplify().n())
    pretty_print(phiS[2][0].subs(z==zABBMinus).subs(C==Cval).canonicalize_radical().full_simplify().n())
    print('----')
    
print("I check that the eigenvector for the bulk is also eigenvector for the boundary, thus instability")
print("Computations are performed directly on the limit point")
kStar = kABB.subs(z==zABBMinus).full_simplify().canonicalize_radical()
LzOnStable = Lz.subs(z==zABBMinus.canonicalize_radical(), k==kStar)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
LzOnStableKer = LzOnStable.transpose().kernel()
phiS = matrix([[LzOnStableKer.basis()[0][0].full_simplify()], \
               [LzOnStableKer.basis()[0][1].full_simplify()], \
               [LzOnStableKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates
bd00 = matrix(SR, q, q)
bd01 = matrix(SR, q, q)

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].subs(z==zABBMinus.canonicalize_radical()).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].subs(z==zABBMinus.canonicalize_radical()).full_simplify().collect(k).coefficient(k, 1)
        
firstColBs = (bd00 + bd01*kStar)*phiS ### First column
for i in range(q):
    pretty_print(firstColBs[i][0].full_simplify())

Anti Bounce back (now correct)


[z == -4/3*C^2 - 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -4/3*C^2 + 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10), kappa == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 - (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

...

0.485822306238185 - 0.499798952579531*I

----
I check that the eigenvector for the bulk is also eigenvector for the boundary, thus instability
Computations are performed directly on the limit point


0

0

0

In [74]:
#### Now we analyze the other boundary conditions which are not always anti bounce back

#### Critical points (-1, 1) and (1, -1) --- For C<0
assume(C<0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print('Perturbation test')
dz = var("dz", latex_name="\\delta z") 
print('For kminus')
pretty_print(taylor(kminus.subs(z==-1-dz), dz, 0, 1))
print('For kplus')
pretty_print(taylor(kplus.subs(z==-1-dz), dz, 0, 1))
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus.full_simplify()

print("Group velocity of the stable mode at z=-1")
pretty_print((ks/z/(taylor(kplus.subs(z==-1-dz), dz, 0, 1) - 1).subs(dz==1)).subs(z==-1).full_simplify())

print("Group velocity of the stable mode at z=1")
pretty_print((-ks/z/ks.derivative(z)).subs(z==1).full_simplify())


LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()], \
               [LzOnStable.transpose().kernel().basis()[0][2].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = -1')
pretty_print(taylor(phiS[1][0], z, -1, -1).factor())
print('Expansion of the third component of phiS around z = -1')
pretty_print(taylor(phiS[2][0], z, -1, 0))
print('Expansion of the second component of phiS around z = 1')
pretty_print(taylor(phiS[1][0], z, 1, 1))
print('Expansion of the third component of phiS around z = 1')
pretty_print(taylor(phiS[2][0], z, 1, 1))



Value of kminus at z=-1


1

Value of kplus at z=-1


1

Perturbation test
For kminus


...

4/3*(C^2 - 1)/(z + 1) + 1

Expansion of the second component of phiS around z = 1


-1/6*(2*C^2 + 1)*(z - 1)/C - 2*C/(z - 1)

Expansion of the third component of phiS around z = 1


2/3*C^2 - 1/3*(C^2 - 1)*(z - 1) + 1/3

In [137]:


#### We construct the eigenvectors for k = 0
print("Basis for the eigenspace of k = 0")
pretty_print(Lz.subs(k==0).transpose().kernel().basis())
phi01 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][2]]]) ## First vector
phi02 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[1][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][2]]]) ## Second vector


#### We start analyzing critical points and Kreiss Lopatinskii determinants
#### (-1, 1)

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)
Bs = matrix(SR, q, q) #### Boundary matrix on the stable subspace


#### Bounce back
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for BB with at z=-1 (Taylor expansion)")
pretty_print(taylor(KLDet, z, -1, -1).factor())

pretty_print(taylor(Bs.inverse(), z, -1, 0))


#### Anti Bounce back
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for ABB with at z=-1 (Taylor expansion)")
pretty_print(taylor(KLDet, z, -1, -1).factor())

pretty_print(taylor(Bs.inverse(), z, -1, 0))


#### Two steps Anti Bounce back
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Two steps ABB with at z=-1 ")
pretty_print(KLDet.subs(z==-1).factor())
pretty_print(Bs.subs(z==-1).simplify_full())
pretty_print(Bs.inverse().subs(z==-1).simplify_full())



#### Extrapolation sigma = 1
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for extrapolation with sigma = 1 at z=-1 ")
pretty_print(taylor(KLDet, z, -1, 0).full_simplify())

print('We check the taylor expansion of 1/(1-abs(ks))')
pretty_print(taylor(1/(1-abs(tmp[0].rhs())), z, -1, -1))## For whatever reason, I cannot use ks (kplus)


#### Extrapolation sigma = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for extrapolation with sigma = 2 at z=-1 (taylor expansion) ")
pretty_print(taylor(KLDet, z, -1, 0).full_simplify())

#### Extrapolation sigma = 3
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
Bs[:, 0] = (bd00 + bd01*ks + bd02*ks**2)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for extrapolation with sigma = 3 at z=-1 (taylor expansion) ")
pretty_print(taylor(KLDet, z, -1, 0).full_simplify())




#### Extrapolation sigma = 4
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
Bs[:, 0] = (bd00 + bd01*ks + bd02*ks**2 + bd03*ks**3)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for extrapolation with sigma = 4 at z=-1 (taylor expansion) ")
pretty_print(taylor(KLDet, z, -1, 0).full_simplify())

#### kinetic dirichlet
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for kinetic dirichlet with at z=-1 (Taylor expansion)")
pretty_print(taylor(KLDet, z, -1, -1).factor())

pretty_print(taylor(Bs.inverse(), z, -1, 0))






#### (1, -1)

#### Bounce back
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for two steps ABB at z=1 ")
pretty_print(KLDet.subs(z==1))


#### Two steps AntiBounce back
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi01 ### Second column
Bs[:, 2] = (bd00)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for BB with at z=1 ")
pretty_print(KLDet.subs(z==1))
pretty_print(Bs.inverse().subs(z==1).simplify_full())

Basis for the eigenspace of k = 0


[
(1, 0, 4/3*C^2 + 2*C + 2/3),
(0, 1, -1)
]

Kreiss Lopatinskii determinant for BB with at z=-1 (Taylor expansion)


-32/9*(5*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 + 3*C^5 - 10*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 30*C^3 - 4*I*sqrt(3)*sqrt(5*C^2 - 8) - 60*C)*(C + 1)^2*(C - 1)^2/((-3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 11*C^6 - 60*C^4 + 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 16)*(z + 1))

[                    0                     0                     0]
[        3/4/(C^2 - 1)                     0        -3/4/(C^2 - 1)]
[1/4*(2*C + 1)/(C - 1)                  -1/2          -3/4/(C - 1)]

...

Kreiss Lopatinskii determinant for two steps ABB at z=1 


0

Kreiss Lopatinskii determinant for BB with at z=1 


-4*C

[         1/2*(2*C + 1)/C                   -1/2/C                        0]
[                  -1/2/C                    1/2/C                        0]
[-1/6*(2*C^2 + 3*C + 1)/C  1/6*(2*C^2 + 6*C + 1)/C                     -1/2]